<a href="https://colab.research.google.com/github/KChamroeun/Application/blob/main/CalendarRender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

heavenly_stem_characters = {
    "Jia": "甲 ឈើ+",
    "Yi": "乙 ឈើ-",
    "Bing": "丙 ភ្លើង+",
    "Ding": "丁 ភ្លើង-",
    "Wu": "戊 ដី+",
    "Ji": "己 ដី-",
    "Geng": "庚 មាស+",
    "Xin": "辛 មាស-",
    "Ren": "壬 ទឹក+",
    "Gui": "癸 ទឹក-"
      }
heavenly_stem_characters_for_year = {
    0: "庚 Geng មាស+",
    1: "辛 Xin 庚 មាស-",
    2: "壬 Ren ទឹក+",
    3: "癸 Gui ទឹក-",
    4: "甲 Jia ឈើ+",
    5: "乙 Yi ឈើ-",
    6: "丙 Bing ភ្លើង+",
    7: "丁 Ding ភ្លើង-",
    8: "戊 Wu ដី+",
    9: "己 Ji ដី-"
}

print(heavenly_stem_characters_for_year)

print(heavenly_stem_characters)





{'Jia': '甲 ឈើ+', 'Yi': '乙 ឈើ-', 'Bing': '丙 ភ្លើង+', 'Ding': '丁 ភ្លើង-', 'Wu': '戊 ដី+', 'Ji': '己 ដី-', 'Geng': '庚 មាស+', 'Xin': '辛 មាស-', 'Ren': '壬 ទឹក+', 'Gui': '癸 ទឹក-'}


In [6]:
import calendar

heavenly_stems = ["Jia", "Yi", "Bing", "Ding", "Wu", "Ji", "Geng", "Xin", "Ren", "Gui"]
earthly_branches = ["Zi", "Chou", "Yin", "Mao", "Chen", "Si", "Wu", "Wei", "Shen", "You", "Xu", "Hai"]
solar_terms_2025 = {  # Example - you'd need the actual dates
    "Start of Spring": "2025-02-03",
    "Rain Water": "2025-02-18",
    # ... and so on for all 24 solar terms
}

def get_day_pillar(year, month, day):
    # This is a simplified placeholder - the actual calculation is complex
    # You'd need a reference date and the 60 Jia Zi cycle logic
    index = (year + month + day) % 60
    heavenly_stem_index = index % 10
    earthly_branch_index = index % 12
    return heavenly_stems[heavenly_stem_index], earthly_branches[earthly_branch_index]

def get_month_pillar(year_heavenly_stem, month):
    # This is also a simplified placeholder - the logic is based on year stem
    # and the fixed Earthly Branch for the month
    year_stem_index = heavenly_stems.index(year_heavenly_stem)
    month_heavenly_stems = ["Bing", "Ding", "Wu", "Ji", "Geng", "Xin", "Ren", "Gui", "Jia", "Yi", "Bing", "Ding"] # Example sequence
    return month_heavenly_stems[(year_stem_index % 5) * 2 + (month - 1) % 12], earthly_branches[month - 1] # Assuming month starts from Yin

def render_bazi_calendar(year):
    cal = calendar.TextCalendar()
    year_stem = get_heavenly_stem_for_year(year) # Assuming you have this function

    for month in range(1, 13): # Looping through months (though Bazi has a different lunar calendar start)
        print(f"\n----- Month {month} -----")
        month_stem, month_branch = get_month_pillar(year_stem, month)
        print(f"Month Pillar: {month_stem} {month_branch}")
        print(cal.formatmonth(year, month))
        for day in range(1, calendar.monthrange(year, month)[1] + 1):
            day_stem, day_branch = get_day_pillar(year, month, day)
            date_str = f"{year}-{month:02d}-{day:02d}"
            solar_term = ""
            for term, date in solar_terms_2025.items(): # This needs to be for the correct year
                if date == date_str:
                    solar_term = f"({term})"
                    break
            print(f"{day}: {day_stem} {day_branch} {solar_term}")

# Example call (will not be fully accurate without the complex logic)
render_bazi_calendar(2025)


----- Month 1 -----
Month Pillar: Wu Zi
    January 2025
Mo Tu We Th Fr Sa Su
       1  2  3  4  5
 6  7  8  9 10 11 12
13 14 15 16 17 18 19
20 21 22 23 24 25 26
27 28 29 30 31

1: Xin Hai 
2: Ren Zi 
3: Gui Chou 
4: Jia Yin 
5: Yi Mao 
6: Bing Chen 
7: Ding Si 
8: Wu Wu 
9: Ji Wei 
10: Geng Shen 
11: Xin You 
12: Ren Xu 
13: Gui Hai 
14: Jia Zi 
15: Yi Chou 
16: Bing Yin 
17: Ding Mao 
18: Wu Chen 
19: Ji Si 
20: Geng Wu 
21: Xin Wei 
22: Ren Shen 
23: Gui You 
24: Jia Xu 
25: Yi Hai 
26: Bing Zi 
27: Ding Chou 
28: Wu Yin 
29: Ji Mao 
30: Geng Chen 
31: Xin Si 

----- Month 2 -----
Month Pillar: Ji Chou
   February 2025
Mo Tu We Th Fr Sa Su
                1  2
 3  4  5  6  7  8  9
10 11 12 13 14 15 16
17 18 19 20 21 22 23
24 25 26 27 28

1: Ren Zi 
2: Gui Chou 
3: Jia Yin (Start of Spring)
4: Yi Mao 
5: Bing Chen 
6: Ding Si 
7: Wu Wu 
8: Ji Wei 
9: Geng Shen 
10: Xin You 
11: Ren Xu 
12: Gui Hai 
13: Jia Zi 
14: Yi Chou 
15: Bing Yin 
16: Ding Mao 
17: Wu Chen 
18: Ji Si (Rain Wat

IndexError: list index out of range